See https://github.com/facebook/Ax/issues/743

In [1]:
%pip install ax-platform

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.7/730.7 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.8 MB/s eta 0:00:00


In [41]:
# %% imports
import numpy as np
import pandas as pd

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, normalize

from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
from ax.modelbridge.registry import Models

from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties

n_train = 7
unique_components = ["filler_A", "filler_B", "resin_A", "resin_B", "resin_C"]
objective_names = ["yield_strength", "elongation"]

X_train, y_train = make_regression(n_samples=n_train, n_features=5, n_informative=5, n_targets=2, noise=0.1, random_state=10)

X_train = MinMaxScaler().fit_transform(X_train)
X_train = normalize(X_train, norm="l1")

y_train[:, 0] = MinMaxScaler(feature_range=(0, 100)).fit_transform(y_train[:, 0].reshape(-1, 1)).ravel()
y_train[:, 1] = MinMaxScaler(feature_range=(0, 5)).fit_transform(y_train[:, 1].reshape(-1, 1)).ravel()

X_train = pd.DataFrame(X_train, columns=unique_components)
print(X_train)

y_train = pd.DataFrame(y_train, columns=objective_names)

,filler_A,filler_B,resin_A,resin_B,resin_C
0,0.263170,0.161356,0.319319,0.177511,0.078643
1,0.233281,0.191723,0.165822,0.409174,0.000000
2,0.067263,0.225302,0.341151,0.186902,0.179382
3,0.242482,0.000000,0.000000,0.363517,0.394001
4,0.000000,0.346443,0.318884,0.272197,0.062476
5,0.293613,0.293613,0.183609,0.208137,0.021027
6,0.268568,0.312212,0.096869,0.000000,0.322350


In [42]:
y_train

array([[ 11.07632106,   1.92667673],
       [ 31.77210828,   1.78073878],
       [ 75.0228475 ,   2.87278749],
       [ 43.80877892,   0.        ],
       [ 44.68872303,   2.62865504],
       [100.        ,   5.        ],
       [  0.        ,   2.04822265]])

In [43]:
# Ax-specific
parameters = [
    {"name": component, "type": "range", "bounds": [0.0, 1.0]}
    for component in unique_components[:-1]
]
parameters

[{'name': 'filler_A', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'filler_B', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'resin_A', 'type': 'range', 'bounds': [0.0, 1.0]},
 {'name': 'resin_B', 'type': 'range', 'bounds': [0.0, 1.0]}]

In [44]:
separator = " + "
composition_constraint = separator.join(unique_components[:-1]) + " <= 1.0"
composition_constraint

'filler_A + filler_B + resin_A + resin_B <= 1.0'

In [45]:
# skip the pseudo-random suggested points by specifying a custom generation strategy
gs = GenerationStrategy(
    steps=[
        # 2. Bayesian optimization step (requires data obtained from previous phase and learns
        # from all data available at the time of each new candidate generation call)
        GenerationStep(
            model=Models.FULLYBAYESIANMOO,
            num_trials=-1,  # No limitation on how many trials should be produced from this step
            max_parallelism=None,  # Parallelism limit for this step, often lower than for Sobol
            # More on parallelism vs. required samples in BayesOpt:
            # https://ax.dev/docs/bayesopt.html#tradeoff-between-parallelism-and-total-number-of-trials
        ),
    ]
)

objectives = {objective_name: ObjectiveProperties(minimize=False) for objective_name in objective_names}

# setup the experiment
ax_client = AxClient(generation_strategy=gs)
ax_client.create_experiment(
    name="dummy",
    parameters=parameters,
    parameter_constraints=[
        composition_constraint,
    ],
    objectives=objectives,
)

[INFO 02-18 04:47:21] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-18 04:47:21] ax.service.utils.instantiation: Due to non-specification, we will use the heuristic for selecting objective thresholds.
[INFO 02-18 04:47:21] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter filler_A. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 02-18 04:47:21] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter filler_B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 02-18 04:47:21] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter resin_A. If that i

In [46]:
# attach the training data
for i in range(n_train):
    ax_client.attach_trial(X_train.iloc[i, :-1].to_dict())
    ax_client.complete_trial(trial_index=i, raw_data={"yield_strength": y_train[i, 0], "elongation": y_train[i, 1]})

[INFO 02-18 04:47:22] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.26317, 'filler_B': 0.161356, 'resin_A': 0.319319, 'resin_B': 0.177511} as trial 0.
[INFO 02-18 04:47:22] ax.service.ax_client: Completed trial 0 with data: {'yield_strength': (11.076321, None), 'elongation': (1.926677, None)}.
[INFO 02-18 04:47:22] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.233281, 'filler_B': 0.191723, 'resin_A': 0.165822, 'resin_B': 0.409174} as trial 1.
[INFO 02-18 04:47:22] ax.service.ax_client: Completed trial 1 with data: {'yield_strength': (31.772108, None), 'elongation': (1.780739, None)}.
[INFO 02-18 04:47:22] ax.service.ax_client: Attached custom parameterization {'filler_A': 0.067263, 'filler_B': 0.225302, 'resin_A': 0.341151, 'resin_B': 0.186902} as trial 2.
[INFO 02-18 04:47:22] ax.service.ax_client: Completed trial 2 with data: {'yield_strength': (75.022848, None), 'elongation': (2.872787, None)}.
[INFO 02-18 04:47:22] ax.service.ax_clien

In [47]:
# produce a *batch* of five next suggested experiments, **be sure to only run this once**
next_experiments, optimization_complete = ax_client.get_next_trials(max_trials=5)
print("next suggested experiments: ", next_experiments)

Sample:  71%|███████   | 542/768 [23:11,  2.57s/it, step size=4.04e-01, acc. prob=0.869]
[INFO 02-18 04:50:30] ax.service.ax_client: Generated new trial 7 with parameters {'filler_A': 0.031971, 'filler_B': 0.968029, 'resin_A': 0.0, 'resin_B': 0.0}.
Warmup:  60%|██████    | 462/768 [00:51, 15.91it/s, step size=2.26e+00, acc. prob=0.788]/usr/local/lib/python3.8/dist-packages/botorch/models/fully_bayesian.py:124: RuntimeWarning: Received a linear algebra error while sampling with Pyro. Adding a jitter of 1e-08 to the covariance matrix and retrying.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/botorch/models/fully_bayesian.py:124: RuntimeWarning: Received a linear algebra error while sampling with Pyro. Adding a jitter of 1e-07 to the covariance matrix and retrying.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/botorch/models/fully_bayesian.py:124: RuntimeWarning: Received a linear algebra error while sampling with Pyro. Adding a jitter of 9.999999999999999e-06 to the 

next suggested experiments:  {7: {'filler_A': 0.031970688369562794, 'filler_B': 0.9680293116304256, 'resin_A': 2.440461820259198e-15, 'resin_B': 5.001874288507975e-15}, 8: {'filler_A': 0.0, 'filler_B': 1.0442941754793412e-15, 'resin_A': 1.0, 'resin_B': 1.0401917373590851e-15}, 9: {'filler_A': 0.07758819690028068, 'filler_B': 0.7061009329668826, 'resin_A': 0.0, 'resin_B': 0.21631087013285272}, 10: {'filler_A': 0.35212900002205166, 'filler_B': 0.47135888511571095, 'resin_A': 1.9744388007959498e-14, 'resin_B': 0.17651211486186685}, 11: {'filler_A': 0.0, 'filler_B': 7.584121535482275e-16, 'resin_A': 0.9999999999999973, 'resin_B': 0.0}}


In [48]:
# note that the model fit is poor because of the toy data and randomly generated objective values
# (i.e. this is what we would expect: a bad fit, because the "true" values are nonsense)
pareto_optimal_parameters = ax_client.get_pareto_optimal_parameters()
print(pareto_optimal_parameters)

Sample: 100%|██████████| 768/768 [01:11, 10.67it/s, step size=2.35e-01, acc. prob=0.966]
[INFO 02-18 05:04:09] ax.service.utils.best_point: Using inferred objective thresholds: [ObjectiveThreshold(elongation >= 2.2548934888450423), ObjectiveThreshold(yield_strength >= 22.222478943939702)], as objective thresholds were not specified as part of the optimization configuration on the experiment.


{5: ({'filler_A': 0.2936132459321941, 'filler_B': 0.2936132459321941, 'resin_A': 0.1836094713241959, 'resin_B': 0.20813736741130703}, ({'elongation': 3.512290639955121, 'yield_strength': 50.4162103784033}, {'elongation': {'elongation': 1.0309070138381722, 'yield_strength': 0.0}, 'yield_strength': {'elongation': 0.0, 'yield_strength': 410.6005483220568}}))}


/usr/local/lib/python3.8/dist-packages/ax/modelbridge/modelbridge_utils.py:831: UserWarning: FYI: The default behavior of `get_pareto_frontier_and_configs` when `transform_outcomes_and_configs` is not specified has changed. Previously, the default was `transform_outcomes_and_configs=True`; now this argument is deprecated and behavior is as if `transform_outcomes_and_configs=False`. You did not specify `transform_outcomes_and_configs`, so this warning requires no action.
  warnings.warn(
